# 模块构造

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10)
)

X = torch.randn(2, 20)
net(X)

tensor([[-0.4326,  0.1334, -0.0564,  0.2212, -0.1222,  0.6305, -0.2560, -0.1907,
         -0.2790, -0.3939],
        [-0.1291,  0.1383,  0.0759,  0.3484, -0.1610,  0.2348,  0.1185, -0.1462,
         -0.1290, -0.1278]], grad_fn=<AddmmBackward>)

定义module

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [4]:
net = MLP()
net(X)

tensor([[-0.3127,  0.1809,  0.0182,  0.2952,  0.1773, -0.3077, -0.1073, -0.4384,
          0.3922, -0.0082],
        [ 0.1832,  0.0563,  0.0144,  0.4135,  0.2618, -0.2807,  0.1946, -0.2416,
          0.1250, -0.1610]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequatial(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 是一个order dictionary，重点按顺序放入，自身作为key
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequatial(nn.Linear(20, 256),
                  nn.ReLU(),
                  nn.Linear(256, 10)
)
net(X)

tensor([[-0.5475, -0.0724, -0.0556, -0.1423,  0.0962, -0.1838,  0.0866,  0.1971,
         -0.0588,  0.2115],
        [ 0.0816, -0.0662, -0.1840, -0.5823, -0.0614, -0.1209, -0.3711, -0.0193,
          0.2050,  0.1749]], grad_fn=<AddmmBackward>)

在正向传播函数执行代码——在init和forward有更多的自定义

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() >1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2428, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1533, grad_fn=<SumBackward0>)

# 参数管理

In [9]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2,4))
net(X)

tensor([[0.0957],
        [0.1966]], grad_fn=<AddmmBackward>)

参数访问

In [10]:
# 权重是一个状态，所以叫做state_dict
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1723,  0.2249, -0.2892,  0.2808,  0.3474,  0.1099,  0.0721,  0.2894]])), ('bias', tensor([-0.1281]))])


目标参数

In [11]:
print(type(net[2].bias)) # parameter可以优化
print(net[2].bias)
print(net[2].bias.data) # 还有.grad访问梯度

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1281], requires_grad=True)
tensor([-0.1281])


In [13]:
# 因为还没做计算，所以grad = None
net[2].bias.grad == None

True

一次访问所有参数

In [15]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
# 0是第一层，1是relu没有参数

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
net.state_dict()['2.bias'].data

tensor([-0.1281])

从嵌套块收集参数

In [18]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # add好处是可以加名字
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), block1())
rgnet(X)

tensor([[0.0000, 0.0000, 0.3920, 0.2477],
        [0.0000, 0.0000, 0.3920, 0.2477]], grad_fn=<ReluBackward0>)

查看已经设计好的网络

In [19]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
)


内置初始化

In [22]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

# apply，对net所有的module，遍历一遍
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0057, -0.0110, -0.0017, -0.0201]), tensor(0.))

In [24]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [25]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0802, -0.1721,  0.4059,  0.3654])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [26]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.8491, -8.7276, -7.5761,  0.0000],
        [ 0.0000, -0.0000, -5.4399, -0.0000]], grad_fn=<SliceBackward>)

In [27]:
# 更直接的方法，把data直接替换
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -7.7276, -6.5761,  1.0000])

小应用-参数绑定，在一些层share parameter，共享权重，就是参数绑定

In [29]:
shared= nn.Linear(8, 8) # 通过提前定义好一个层，重复使用
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(), shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 自定义层
其实和自定义网络没有本质区别，层也是一个nn.Module的子类

构造一个没有任何参数的自定义层

In [30]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到构建更复杂的模型中

In [31]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4,8))
Y.mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

带参数的图层

In [32]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # Parameter目的：声明需要计算梯度，同时给予一个合适的名字
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))

    def forward(self, X):
        # Parameter需要.data去访问参数
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)


dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[ 0.9959, -0.9931,  1.9899],
        [ 0.0781,  0.1961, -1.2133],
        [-0.6171, -0.7141,  1.9012],
        [-0.3853, -0.9047,  1.1611],
        [ 1.4197,  0.1081,  0.2144]], requires_grad=True)

In [33]:
dense(torch.rand(2, 5))

tensor([[0.8899, 0.8416, 1.1059],
        [1.0684, 0.0907, 2.8331]])

In [36]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[1.4725],
        [0.8605]])

# 读写文件

加载和保持张量

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'data/x-file')

x2 = torch.load("data/x-file")
x2

tensor([0, 1, 2, 3])

存一个张量列表，然后把他们读回内存

In [2]:
y = torch.zeros(4)
torch.save([x, y], 'data/x-files')

x2, y2 = torch.load('data/x-files')
[x, y]

[tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.])]

写入或读取从字符串映射到张量的字典

In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'data/mydict')
mydict2 = torch.load('data/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

In [6]:
# state_dict()得到所有数据
torch.save(net.state_dict(), 'model/mlp.params')

In [7]:
# 首先要声明MLP
clone = MLP()
clone.load_state_dict(torch.load("model/mlp.params"))
# 
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)

In [8]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])